In [1]:
# pip install fastf1

In [2]:
import os
import sys
import fastf1
try:
    fastf1.Cache.enable_cache(sys.path[0]+"/fastf1_cache") 
except:
    os.makedirs(sys.path[0]+"/fastf1_cache")
    fastf1.Cache.enable_cache(sys.path[0]+"/fastf1_cache") 
from fastf1 import plotting
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.collections import LineCollection
import datetime
import seaborn as sns
sns.set_style("darkgrid")
import pandas as pd
import numpy as np
import math
from windrose import WindroseAxes
pd.set_option('display.max_columns', None)

import plotly.express as px
import plotly.graph_objects as go

## Singapore Grand Prix 2023 - Race
Analysis adapted from Tracing Insights for Singapore 2023, with some aspects of data processing to suit the Game Theory project data requirements.

In [3]:
# query dataset for Singapore 2023
# https://github.com/TracingInsights/formula1-dataanalysis/tree/main

race = fastf1.get_session(2023, "Singapore", 'R')
race.load(weather=True)

core           INFO 	Loading data for Singapore Grand Prix - Race [v3.3.2]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	No lap data for driver 18
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 18)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['55', '4', '44', '16', '1',

In [4]:
# exploratory data analysis

df = race.laps
df = df.sort_values(by=['LapNumber','Position'], ascending=[False, True]).reset_index(drop=True)
df.LapTime = df.LapTime.fillna(df['Sector1Time']+df['Sector2Time']+df['Sector3Time'])
df.LapTime = df.LapTime.dt.total_seconds()
df

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 02:49:21.852000,SAI,55,99.958,62.0,2.0,NaT,NaT,0 days 00:00:29.012000,0 days 00:00:43.101000,0 days 00:00:27.845000,0 days 02:48:10.906000,0 days 02:48:54.007000,0 days 02:49:21.852000,294.0,262.0,249.0,NaN,False,HARD,42.0,True,Ferrari,0 days 02:47:41.894000,2023-09-17 13:48:42.405,12,1.0,False,,False,True
1,0 days 02:49:22.629000,NOR,4,99.677,62.0,2.0,NaT,NaT,0 days 00:00:28.523000,0 days 00:00:43.224000,0 days 00:00:27.930000,0 days 02:48:11.520000,0 days 02:48:54.744000,0 days 02:49:22.674000,313.0,278.0,248.0,289.0,False,HARD,42.0,True,McLaren,0 days 02:47:42.952000,2023-09-17 13:48:43.463,12,2.0,False,,False,True
2,0 days 02:49:23.092000,HAM,44,99.347,62.0,3.0,NaT,NaT,0 days 00:00:28.765000,0 days 00:00:42.817000,0 days 00:00:27.765000,0 days 02:48:12.531000,0 days 02:48:55.348000,0 days 02:49:23.113000,317.0,279.0,231.0,304.0,False,MEDIUM,18.0,True,Mercedes,0 days 02:47:43.745000,2023-09-17 13:48:44.256,12,3.0,False,,False,True
3,0 days 02:49:43.016000,LEC,16,100.946,62.0,2.0,NaT,NaT,0 days 00:00:29.131000,0 days 00:00:43.503000,0 days 00:00:28.312000,0 days 02:48:31.210000,0 days 02:49:14.713000,0 days 02:49:43.025000,NaN,261.0,244.0,NaN,False,HARD,42.0,True,Ferrari,0 days 02:48:02.070000,2023-09-17 13:49:02.581,12,4.0,False,,False,True
4,0 days 02:49:43.290000,VER,1,98.197,62.0,2.0,NaT,NaT,0 days 00:00:28.493000,0 days 00:00:41.975000,0 days 00:00:27.729000,0 days 02:48:33.582000,0 days 02:49:15.557000,0 days 02:49:43.286000,293.0,267.0,252.0,287.0,False,MEDIUM,22.0,True,Red Bull Racing,0 days 02:48:05.093000,2023-09-17 13:49:05.604,12,5.0,False,,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,0 days 01:04:36.357000,PIA,81,111.927,1.0,1.0,NaT,NaT,NaT,0 days 00:00:46.249000,0 days 00:00:28.945000,NaT,0 days 01:04:07.460000,0 days 01:04:36.350000,286.0,264.0,244.0,246.0,False,MEDIUM,1.0,False,McLaren,0 days 01:02:44.148000,2023-09-17 12:03:44.659,12,15.0,False,,False,False
1084,0 days 01:04:37.507000,ALB,23,113.077,1.0,1.0,NaT,NaT,NaT,0 days 00:00:47.752000,0 days 00:00:28.526000,NaT,0 days 01:04:09.022000,0 days 01:04:37.553000,287.0,265.0,241.0,231.0,False,MEDIUM,1.0,True,Williams,0 days 01:02:44.148000,2023-09-17 12:03:44.659,12,16.0,False,,False,False
1085,0 days 01:04:38.490000,SAR,2,114.060,1.0,1.0,NaT,NaT,NaT,0 days 00:00:48.432000,0 days 00:00:28.675000,NaT,0 days 01:04:09.881000,0 days 01:04:38.475000,301.0,265.0,240.0,245.0,False,MEDIUM,1.0,True,Williams,0 days 01:02:44.148000,2023-09-17 12:03:44.659,12,17.0,False,,False,False
1086,0 days 01:04:39.144000,BOT,77,114.714,1.0,1.0,NaT,NaT,NaT,0 days 00:00:47.958000,0 days 00:00:29.043000,NaT,0 days 01:04:10.163000,0 days 01:04:39.147000,282.0,259.0,237.0,237.0,False,HARD,1.0,True,Alfa Romeo,0 days 01:02:44.148000,2023-09-17 12:03:44.659,12,18.0,False,,False,False


## 1. Analysis of Lap Time Distribution by Driver

In [5]:
# for color palette
driver_color = {}
for index,lap in df.iterrows():
    driver = lap['Driver']
    driver_color[driver] = fastf1.plotting.driver_color(driver)

# # Plot box whisker plots for lap time distribution of each driver
# plt.figure(figsize=(20,10))
# sns.boxplot(data = df, x = df.Driver, y=df.LapTime, palette=driver_color)

# plt.ylim(min(df.LapTime)-0.5, df.LapTime.median()+10)
# plt.ylabel('Lap Time (s)')
# plt.title('Lap Time Distribution for Each Driver in Singapore 2023',  fontsize=20)
# plt.show()

In [6]:
color_discrete_map = {driver: color for driver, color in driver_color.items() if driver in df['Driver'].unique()}

fig = px.box(df, x='Driver', y='LapTime',
             title='Lap Time Distribution for Each Driver in Singapore 2023',
             labels={'LapTime': 'Lap Time (s)', 'Driver': 'Driver'},
             color='Driver',
             color_discrete_map=color_discrete_map,
             template="plotly_dark")

fig.update_layout(yaxis=dict(range=[min(df['LapTime'])-0.5, df['LapTime'].median()+10]))
fig.update_layout(width=2000, height=1000)

fig.show()

#### Observation 1:
SAI (FERRARI) and HAM (MERCEDES), together with NOR (MCLAREN) are in the top 15th percentile of drivers, i.e., top 3.
<br><br>

#### Changes in lap times by the Top 15 drivers

In [7]:
# top 3 Drivers (based on final result)
all_drivers = list(race.results.Abbreviation.iloc[:20])

# data cleaning 
df_all_drivers = df.loc[df.Driver.isin(all_drivers),['LapTime','LapNumber','Driver']]
df_all_drivers = df_all_drivers.reset_index(drop=True)

# plt.figure(figsize=(20,6))
# sns.lineplot(df_all_drivers, x=df_all_drivers['LapNumber'], y=df_all_drivers['LapTime'], marker = "o", hue=df_all_drivers['Driver'], palette = driver_color)
# plt.ylabel('Lap Time (s)')
# plt.xlabel('Laps')
# plt.title('Lap Time of All Drivers across Race \n Singapore 2023',  fontsize=20)
# plt.ylim(min(df_all_drivers.LapTime)-0.5, df_all_drivers.LapTime.median()+5)
# plt.show()

In [8]:
color_discrete_map = {driver: color for driver, color in driver_color.items() if driver in df_all_drivers['Driver'].unique()}

fig = px.line(df_all_drivers, x='LapNumber', y='LapTime', color='Driver',
              title='Lap Time of All Drivers across Race - Singapore 2023',
              labels={'LapTime': 'Lap Time (s)', 'LapNumber': 'Laps', 'Driver': 'Driver'},
              color_discrete_map=color_discrete_map,
              template="plotly_dark",
              markers=True) # Adds markers to the line plot

# Setting the y-axis limits
fig.update_yaxes(range=[min(df_all_drivers['LapTime'])-0.5, df_all_drivers['LapTime'].median()+5])

# Adjusting the plot size
fig.update_layout(width=2000, height=600)

fig.show()

In [9]:
# top 3 Drivers (based on final result)
top_3_drivers = list(race.results.Abbreviation.iloc[:3])

# data cleaning 
df_top3 = df.loc[df.Driver.isin(top_3_drivers),['LapTime','LapNumber','Driver']]
df_top3 = df_top3.reset_index(drop=True)

# plt.figure(figsize=(20,6))
# sns.lineplot(df_top3, x=df_top3['LapNumber'], y=df_top3['LapTime'], marker = "o", hue=df_top3['Driver'], palette = driver_color)
# plt.ylabel('Lap Time (s)')
# plt.xlabel('Laps')
# plt.title('Lap Time of Top 3 Drivers across Race \n Singapore 2023',  fontsize=20)
# plt.ylim(min(df_top3.LapTime)-0.5, df_top3.LapTime.median()+5)
# plt.show()

In [10]:
color_discrete_map = {driver: color for driver, color in driver_color.items() if driver in df_top3['Driver'].unique()}

fig = px.line(df_top3, x='LapNumber', y='LapTime', color='Driver',
              title='Lap Time of Top 3 Drivers across Race - Singapore 2023',
              labels={'LapTime': 'Lap Time (s)', 'LapNumber': 'Laps', 'Driver': 'Driver'},
              color_discrete_map=color_discrete_map,
              template="plotly_dark",
              markers=True) # Adds markers to the line plot for each data point

fig.update_yaxes(range=[min(df_top3['LapTime'])-0.5, df_top3['LapTime'].median()+5])
fig.update_layout(width=2000, height=600)

fig.show()


#### Observation 2:
Race pace of HAM on new Medium tyres is significantly faster than those of NOR and SAI on used Hard tyres between Laps 45 to 57. This has allowed HAM to close the gap to SAI. Yet, we see that there is a bunching up of lap times from Lap 58 onwards despite HAM being on faster Medium tyres. This suggests that race pace is not the sole factor in deciding the win, with defensive strategy in play by SAI to prevent HAM from overtaking.

Additionally, the race pace of the Top 3 is at least a second faster than the rest of the pack - thus lap time distribution when accounting for the three drivers will be skewed.

## 2. Analysis of Lap Time Distribution by Tyre Strategy

#### Tyre strategy of each driver

In [11]:
# fig, ax = plt.subplots(figsize=(20, 10))
# plt.title('Tyre Strategy \n'+"Singapore 2023",  fontsize=30)
# plt.xlabel('Laps')
# plt.grid(False)

# Tyre Stint
compound_color = {key: value for key,value in fastf1.plotting.COMPOUND_COLORS.items()}
tyre_stint = df.groupby(['Driver','Stint','Compound','FreshTyre']).agg({'LapNumber': 'min', 'TyreLife': 'count'}).reset_index()

for drv in list(race.results.Abbreviation)[::-1]:
    driver_stints = tyre_stint[tyre_stint['Driver']== drv]

    # for idx, row in driver_stints.iterrows():
    #     plt.barh(
    #         y=drv,
    #         width=row["TyreLife"],
    #         left=row['LapNumber']-1,
    #         color=compound_color[row.Compound],
    #         edgecolor="black",
    #         fill=True,
    #         alpha = 0.4 if not row.FreshTyre else 1,
    #         label = row.Compound
    #     )

    #     if not row['LapNumber'] == 1.0:
    #         plt.text(row['LapNumber']-1.25, drv, round(row['LapNumber']-1), fontweight='extra bold', backgroundcolor='black', color = 'white')

    # plt.text(df.LapNumber.max()+1, drv, driver_stints['TyreLife'].sum(), fontweight='extra bold', backgroundcolor='black', color = 'white')

In [12]:
fig = go.Figure()

# Tyre Stint
compound_color = {key: value for key, value in fastf1.plotting.COMPOUND_COLORS.items()}
tyre_stint = df.groupby(['Driver', 'Stint', 'Compound', 'FreshTyre']).agg({'LapNumber': 'min', 'TyreLife': 'count'}).reset_index()

# List of drivers
drivers = list(race.results.Abbreviation)[::-1]

for drv in drivers:
    driver_stints = tyre_stint[tyre_stint['Driver'] == drv]

    # Loop through each stint of the driver
    for idx, row in driver_stints.iterrows():
        # Add bar trace for each stint
        fig.add_trace(go.Bar(
            y=[drv],
            x=[row["TyreLife"]],
            orientation='h',
            base=row['LapNumber'] - 1,
            marker=dict(color=compound_color[row.Compound]),
            opacity=0.4 if not row.FreshTyre else 1,
            name=row.Compound,
            legendgroup=row.Compound,  # Group bars by Compound for legend
            showlegend=False  # Only show one legend entry for each Compound
        ))

        # Add annotation for lap number
        if not row['LapNumber'] == 1.0:
            fig.add_annotation(
                x=row['LapNumber'] - 1.25,
                y=drv,
                text=str(round(row['LapNumber'] - 1)),
                font=dict(color='white', size=10),
                showarrow=False,
                bgcolor='black'
            )

    # Add annotation for total tyre life
    fig.add_annotation(
        x=df.LapNumber.max() + 1,
        y=drv,
        text=str(driver_stints['TyreLife'].sum()),
        font=dict(color='white', size=10),
        showarrow=False,
        bgcolor='black'
    )

fig.update_layout(
    title='Tyre Strategy\nSingapore 2023',
    title_font_size=30,
    xaxis_title='Laps',
    yaxis=dict(title='Driver'),  
    legend_title_text='Compound',
    # plot_bgcolor='white',  
    barmode='stack',  # Stack bars horizontally
    height=600,
    width=1000,
    template="plotly_dark"
)

fig.show()

#### Lap time distribution of each tyre compound

In [13]:
# # Lap Time distribution by tyre compound
# compound_color = {key: value for key,value in fastf1.plotting.COMPOUND_COLORS.items()}

# # Plot box whisker plots for lap time distribution of each driver
# plt.figure(figsize=(20,10))
# sns.boxplot(data = df, x = df.Compound, y=df.LapTime, palette=compound_color)

# plt.ylim(min(df.LapTime)-0.5, df.LapTime.median()+10)
# plt.ylabel('Lap Time (s)')
# plt.title('Lap Time Distribution for each Tyre Compound in Singapore 2023',  fontsize=20)
# plt.show()

In [14]:
compound_color = fastf1.plotting.COMPOUND_COLORS

fig = px.box(df, x='Compound', y='LapTime', color='Compound',
             color_discrete_map=compound_color,
             title='Lap Time Distribution for each Tyre Compound in Singapore 2023',
             template="plotly_dark")
fig.update_layout(xaxis_title='Tyre Compound', yaxis_title='Lap Time (s)',
                  font=dict(size=18))

# fig.update_yaxes(range=[95, 110])  # Set the y-axis range

fig.show()



#### Changes to lap time throughout the race by the Top 3 drivers

In [15]:
# # Get lap times for each driver on each compound
# # only selected laptimes with normal track conditions (Track Status = 1)
# # use 15th percentile (top 3/20 fastest pace) given the quality of mercedes and ferrari as a team & driver race style (Sainz, Hamilton)
# plt.figure(figsize=(20,6))

laptime_by_compound = []

for compound in set(df.Compound):
    exec(f"average_laptime_per_tyrelife_{compound} = {{}}")
    data_tyre = df.pick_tyre(compound)
    for tyrelife in range(1,int(data_tyre.TyreLife.max())+1):
        exec(f"average_laptime_per_tyrelife_{compound}[tyrelife] = data_tyre.loc[(df['TyreLife'] == float(tyrelife)) & (df['TrackStatus'] == '1'), ['LapTime']].quantile(0.15)")
    exec(f"key = average_laptime_per_tyrelife_{compound}.keys()")
    exec(f"value = average_laptime_per_tyrelife_{compound}.values()") 
    color = fastf1.plotting.COMPOUND_COLORS[compound] if compound != 'HARD' else "000000" # change HARD tyre from white to black
    # plt.plot(key, value, color = color, marker='o')
    exec(f"result = average_laptime_per_tyrelife_{compound}")
    name = str(compound)
    laptime_by_compound.append([name, result])

# plt.ylabel('Lap Time (s)')
# plt.xlabel('Tyre Life')
# plt.title('85th Percentile (Top 3 drivers) Lap Time across Tyre Life \n'+"Singapore 2023",  fontsize=20)
# plt.show()  

In [16]:


# Assuming 'df' is your DataFrame and it has been correctly loaded with the relevant data.

# Filter the DataFrame for normal track conditions only
df_normal = df[df['TrackStatus'] == '1']

# Unique compounds in the dataset
compounds = df_normal['Compound'].unique()

# Prepare a dictionary for Plotly colors (updating HARD tyre color to black)
compound_colors = {
    'SOFT': 'red',  # Example color, replace with actual color codes
    'MEDIUM': 'yellow',  # Example color, replace with actual color codes
    'HARD': 'white'  # Setting HARD to black
    # Add other compounds as necessary
}

# Initialize a list to collect Plotly go.Scatter objects for plotting
traces = []

for compound in compounds:
    compound_df = df_normal[df_normal['Compound'] == compound]
    average_laptime_per_tyrelife = {}

    for tyrelife in range(1, int(compound_df['TyreLife'].max()) + 1):
        tyrelife_df = compound_df[compound_df['TyreLife'] == tyrelife]
        # Calculate 15th percentile lap time
        percentile_15th = tyrelife_df['LapTime'].quantile(0.15)
        average_laptime_per_tyrelife[tyrelife] = percentile_15th

    # Extracting keys and values for Plotly
    tyre_life = list(average_laptime_per_tyrelife.keys())
    lap_times = list(average_laptime_per_tyrelife.values())
    
    # Creating a scatter plot for each compound
    trace = go.Scatter(x=tyre_life, y=lap_times, mode='lines+markers',
                       name=compound, marker=dict(color=compound_colors.get(compound, 'grey')))
    traces.append(trace)

# Plotting
fig = go.Figure(data=traces)
fig.update_layout(title='85th Percentile Lap Time across Tyre Life <br>Singapore 2023',
                  xaxis_title='Tyre Life',
                  yaxis_title='Lap Time (s)',
                  legend_title='Compound',
                  template="plotly_dark")
fig.show()


#### Extrapolating and modelling tyre lap time degradation with tyre age

In [17]:
# dct[1].values()

In [18]:
# helper function to get quadratic correlation

def get_quadratics(dct):
    keys = list(dct[1].keys())
    vals = list(dct[1].values())
    n = []

    for i in range(len(vals)):
        n.append(vals[i].LapTime)

    new_dict = dict(zip(keys,n))
    cleaned_dict = {key: value for key, value in new_dict.items() if not math.isnan(value)}
    # cleaned_dict = {key: value for key, value in new_dict.items() if not math.isnan(value) and value <= 110} # Threshold Above 110 for now after checking box plot

    
    

    # fit all tyre age and laptime into a quadratic correlation
    a,b,c = np.polyfit(list(cleaned_dict.keys()), list(cleaned_dict.values()),2)
    print(dct[0],a,b,c)
    return a,b,c

In [19]:
# # plot correlated tyre age and lap time for Singapore GP

# fig, ax = plt.subplots(figsize=(20, 10))
# plt.title('Top 3 Drivers Tyre Age and Lap Time \n'+"Singapore 2023",  fontsize=30)
# plt.xlabel('Tyre Age')
# plt.ylabel('Lap Time (s)')

# tyre_age = list(range(63))

# for dct in laptime_by_compound:
#     compound = dct[0]       #compound type
#     a,b,c = get_quadratics(dct)
#     lap_time = []
#     for x in tyre_age:
#         lap_time.append(a*x**2 + b*x + c)
#     color = fastf1.plotting.COMPOUND_COLORS[compound] if compound != 'HARD' else "000000" # change HARD tyre from white to black
#     plt.plot(tyre_age, lap_time, color = color, marker='o')

# plt.show()


In [20]:
# Example COMPOUND_COLORS dictionary for Plotly, adjust as necessary
COMPOUND_COLORS = {
    'SOFT': 'red',
    'MEDIUM': 'yellow',
    'HARD': 'white',  
}

fig = go.Figure()

tyre_age = list(range(63))

for dct in laptime_by_compound:
    compound = dct[0]  # Extract compound type
    a, b, c = get_quadratics(dct)  # Extract quadratic coefficients
    
    lap_time = [a*x**2 + b*x + c for x in tyre_age]
    
    color = COMPOUND_COLORS.get(compound, 'gray')
    
    fig.add_trace(go.Scatter(x=tyre_age, y=lap_time, mode='lines+markers',
                             name=compound, line=dict(color=color)))

fig.update_layout(
    title='Top 3 Drivers Tyre Age and Lap Time \nSingapore 2023',
    title_font_size=30,
    xaxis_title='Tyre Age',
    yaxis_title='Lap Time (s)',
    template="plotly_dark",
    xaxis=dict(range=[0, 63]),  # Adjust axis limits if necessary
)

fig.show()


MEDIUM 0.01373902477042166 -0.5445335863621549 103.66638621613211
SOFT 0.010632527089783267 -0.20480955237358175 99.32181409958719
HARD 0.009342750556306952 -0.49584233521158183 105.26568511257031


## 3. Pit stop timings

In [21]:
# average time (sec) spent at pit stop by team

# extract pit enter and exit time
pitstop = df.loc[(df.LapNumber != 1.0) & (df.PitInTime.combine_first(df.PitOutTime).notnull()) , ['Team','Driver','LapNumber','PitOutTime', 'PitInTime']].sort_values(by=['Team','Driver','LapNumber']).reset_index()

# calculate time spent for each pit stop
pitstop['pittime'] = (pitstop.PitOutTime - pitstop.PitInTime.shift(1)).dt.total_seconds()

# aggregrate for each team
pitstop_team_data = pitstop.groupby(['Team'])['pittime'].mean().sort_values().round(3)

# for colour palette
team_color = {}
for team in pitstop_team_data.index:
    team_color[team] = fastf1.plotting.driver_color(driver)

# plt.figure(figsize=(20, 6))
# sns.barplot(x=pitstop_team_data.index, y=pitstop_team_data.values, palette=team_color, edgecolor='black')
# plt.ylabel('Time Taken (s)')
# plt.xlabel('Team')
# plt.ylim(19,30)
# # to add data labels
# for i in range(len(pitstop_team_data)):
#     plt.text(i, min(pitstop_team_data[i],30)+0.1, pitstop_team_data[i], ha = 'center')  
# plt.title('Average Time Spent on Pit Stops in Singapore\n',  fontsize=20)

In [22]:
import plotly.graph_objects as go
import numpy as np

# Assuming 'teams' is a list of team names in the same order as 'pitstop_team_data' values.
# 'pitstop_team_data' contains the average pit stop times for each team.
# 'team_color' is a dictionary with team names as keys and color codes as values.

# Convert 'pitstop_team_data' to a list if it's not already (assuming it might be a NumPy array)
times = pitstop_team_data.tolist() if isinstance(pitstop_team_data, np.ndarray) else pitstop_team_data

colors = [team_color[team] for team in pitstop_team_data.index]

fig = go.Figure()

# Create the bar chart
fig.add_trace(go.Bar(
    x=pitstop_team_data.index,
    y=times,
    marker_color=colors, # Set the bar colors
    marker_line_color='black', # Edge color of the bars
    marker_line_width=1.5, # Edge line width
))

# Add data labels for each bar
for i, time in enumerate(times):
    fig.add_annotation(
        x=pitstop_team_data.index[i],
        # y=min(time, 30) + 0.1,
        text=str(time),
        showarrow=False,
        yshift=10,
    )

# Update the layout
fig.update_layout(
    title='Average Time Spent on Pit Stops in Singapore',
    title_x=0.5, # Center the title
    xaxis_title='Team',
    yaxis_title='Time Taken (s)',
    # yaxis_range=[19,35], # Set the y-axis limits
    template='plotly_dark', # Choose a theme
    font=dict(size=12), # General font size
    title_font=dict(size=20), # Title font size
)

fig.show()


In [23]:
# average time (sec) spent at pit stop by driver
pitstop_driver_data = pitstop.groupby(['Driver'])['pittime'].mean().sort_values().round(3)

# plt.figure(figsize=(20, 10))
# sns.barplot(x=pitstop_driver_data.index, y=pitstop_driver_data.values, palette=driver_color, edgecolor='black')
# plt.ylabel('Time Taken (s)')
# plt.xlabel('Team')
# plt.ylim(pitstop_driver_data[0]-1, pitstop_driver_data[10]+5)
# # to add data labels
# for i in range(len(pitstop_driver_data)):
#     plt.text(i, min(pitstop_driver_data[i],pitstop_driver_data[10]+5)+0.1, pitstop_driver_data[i], ha = 'center')  
# plt.title('Total Time Spent on Pit Stops (Driver) in Singapore 2023',  fontsize=20)

In [24]:
import plotly.express as px
import pandas as pd

# Assuming pitstop_driver_data is your Series with average pit times per driver
# Example:
# pitstop_driver_data = pd.Series({
#     'Driver A': 22.5, 
#     'Driver B': 23.0, 
#     'Driver C': 21.5
# })

# Assuming you have a dictionary mapping drivers to colors
# For the sake of an example, this is a simple color map. Replace it with your actual driver_color map.
# driver_color = {
#     'Driver A': 'red', 
#     'Driver B': 'blue', 
#     'Driver C': 'green'
# }

# Map the driver names to their respective colors using the driver_color dictionary
colors = [driver_color[driver] for driver in pitstop_driver_data.index]

fig = px.bar(
    pitstop_driver_data, 
    y=pitstop_driver_data.values, 
    x=pitstop_driver_data.index, 
    text=pitstop_driver_data.values.round(3),
    color=colors,  # This assigns colors based on the mapped driver colors
    color_discrete_map="identity"  # This tells Plotly to use the given colors as-is
)

# Update layout for aesthetics similar to the original matplotlib plot
fig.update_traces(texttemplate='%{text}s', textposition='outside')
fig.update_layout(
    title_text='Total Time Spent on Pit Stops (Driver) in Singapore 2023',
    xaxis_title="Driver",
    yaxis_title="Time Taken (s)",
    uniformtext_minsize=8, 
    uniformtext_mode='hide',
    yaxis=dict(range=[pitstop_driver_data.min()-1, pitstop_driver_data.iloc[10]+5])
)

fig.show()

